# Conditioning, stability, and accuracy

Understanding why some numerical calculations go awry. 



## A problematic problem

Problem 5 from the homework asks for the root of $f(x) = x^3 - 2x^2 + 4x/3 - 8/27$. Let's investigate this problem with the secant method (`secantsearch` code repeated here from the previous lecture). 

In [6]:
function secantsearch(f, a::AbstractFloat, b::AbstractFloat, diagnostics=false) 
# return solution x of f(x) == 0, with a,b serving as initial guesses
    
    fa = f(a)           # cache values of f(a), f(b)     
    fb = f(b)
    
    # set c to better of a,b (and fc, too)
    c,fc = abs(fa) < abs(fb) ? (a,fa) : (b,fb)
    
    floattype = typeof((a+b)/2) # this'll eval to bigger of a,b types
    ϵ = eps(floattype)  # machine epsilon
    n = 1               # iteration count
    N = 1024            # max iterations
    
    # start vector for storing root iterates xₙ, assign x₁ = c
    x = zeros(floattype, 1)
    x[1] = c
    
    # stop when f(c) is small, |b-a| is small relative to a and b
    # or when we reach maximium number of iterations
    while abs(fc) > 10ϵ && abs(b-a)/(abs(a)+abs(b)) > 10ϵ && n < N
    
        # compute new values for c, fc by secant approximation
        c = a - fa*(b-a)/(fb-fa)
        fc = f(c)
        f′ = (fb-fa)/(b-a)
        
        # print diagnostics if they're turned on
        diagnostics && println("n = $n, c = $c, f(c) = $(f(c))")
        
        # error and exit if f(c) is bigger than both f(a) and f(b)
        if abs(fc) > abs(fa) && abs(fc) > abs(fb)
            println("secantsearch error: f(c) is bigger than f(a), f(b)")
            break # break out of while loop, don't store bad value c
        end
        
        # replace whichever of a, b has bigger f(a), f(b)
        if abs(fa) > abs(fb)
            a,fa = c,fc
        else
            b,fb = c,fc
        end      
        
        # store c in x, then increment iteration counter
        push!(x,c)   
        n += 1
    end
    
    # return c and vector x
    c, x
    
end

secantsearch (generic function with 2 methods)

In [2]:
f(x) = x.^3 - 2*x.^2 + 4/3*x - 8/27

f (generic function with 1 method)

In [7]:
r,x = secantsearch(f, 0.0, 1.0, true);

n = 1, c = 0.8888888888888891, f(c) = 0.010973936899862813
n = 2, c = 0.8421052631578949, f(c) = 0.005399772129616065
n = 3, c = 0.796785304247992, f(c) = 0.002203020416244428
n = 4, c = 0.7655533526628087, f(c) = 0.0009669710403885645
n = 5, c = 0.7411203540240782, f(c) = 0.0004127229640434882
n = 6, c = 0.7229262275146178, f(c) = 0.000178069284412663
n = 7, c = 0.7091194332075769, f(c) = 7.650996304064961e-5
n = 8, c = 0.6987180509624937, f(c) = 3.292610616567604e-5
n = 9, c = 0.6908601641064995, f(c) = 1.4161066591356075e-5
n = 10, c = 0.6849301974472575, f(c) = 6.091920437989451e-6
n = 11, c = 0.6804532819965489, f(c) = 2.6204324846679228e-6
n = 12, c = 0.6770739084149917, f(c) = 1.1272154389896372e-6
n = 13, c = 0.6745228512285835, f(c) = 4.848808504709368e-7
n = 14, c = 0.6725971277812471, f(c) = 2.085765059556266e-7
n = 15, c = 0.6711434384851674, f(c) = 8.972115939087644e-8
n = 16, c = 0.6700460819689645, f(c) = 3.859443598042134e-8
n = 17, c = 0.6692177117354121, f(c) = 1.6601

Hmmm, that's interesting. The secant method took 36 iterations before stopping on this $f(x)$, when it only took 7 iterations for $f(x) = x^2 - 4$. Also, `c` is still changing in the 5th digit between the last two iterations! It looks like we only have 5 or 6 digits of accuracy in our solution $r \approx 0.66667$. That decimal expansion is pretty suggestive, though. Let's check $f(2/3)$

In [34]:
f(2/3)

0.0

Indeed, the floating-point computation of `f(2/3)` is zero, exactly! So why can't the secant search find the solution, $r=2/3$ to floating-point accuracy? i.e. `r=0.66666666666666667`?

# Investigating limits of accuracy 

To understand the difficulty of solving $f(x)=0$, let's look at $f(x)$ in the neighborhood of the root $x=2/3$. Examine $f(x+\epsilon)$ for $\epsilon = 10^{-15}$ through $10^{-1}$.

In [63]:
ϵ = logspace(-15,-1,15)

15-element Array{Float64,1}:
 1.0e-15
 1.0e-14
 1.0e-13
 1.0e-12
 1.0e-11
 1.0e-10
 1.0e-9 
 1.0e-8 
 1.0e-7 
 1.0e-6 
 1.0e-5 
 0.0001 
 0.001  
 0.01   
 0.1    

In [64]:
f(2/3+ϵ)

15-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 8.88178e-16
 9.99978e-13
 1.0e-9     
 1.0e-6     
 0.001      

Wha...? $f(2/3 + \epsilon)$ evaluates to zero for $\epsilon$ from $10^{-15}$ to $10^{-6}$ !!!

If  $f(2/3 + 10^{-5})$ evaluates to zero, then it's no suprise that we can't determine the $x=2/3$ root of $f(x)$ to more than five digits of accuracy.

Let's plot $f(x)$ around the root to get a sense of its behavior.

In [28]:
using Plots
gr()
x = linspace(0,1.3)
plot(x,f(x), label="f(x)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1436,375.813 50.1879,353.335 61.2322,332.645 72.2765,313.668 83.3208,296.332 94.3652,280.561 105.409,266.282 116.454,253.421 127.498,241.903 138.542,231.654 
 149.587,222.601 160.631,214.669 171.675,207.784 182.72,201.871 193.764,196.858 204.808,192.669 215.853,189.231 226.897,186.47 237.941,184.31 248.986,182.679 
 260.03,181.502 271.074,180.706 282.118,180.215 293.163,179.956 304.207,179.854 315.251,179.837 326.296,179.828 337.34,179.756 348.384,179.544 359.429,179.12 
 370.473,178.408 381.517,177.336 392.562,175.829 403.606,173.812 414.65,171.212 425.695,167.954 436.739,163.965 447.783,159.17 458.828,153.495 469.872,146.867 
 480.916,139.21 491.96,130.451 503.005,120.516 514.049,109.331 525.093,96.821 536.138,82.9126 547.182,67.5315 558.226,50.6036 569.271,32.0548 580.315,11.811 
 
 "/>
 
 
 
 
 f(x)

In [65]:
g(x) = (x-2/3).^3

g (generic function with 1 method)

In [67]:
g(2/3 + ϵ)

15-element Array{Float64,1}:
 9.97604e-46
 9.97604e-43
 1.00093e-39
 9.99934e-37
 1.0e-33    
 1.0e-30    
 1.0e-27    
 1.0e-24    
 1.0e-21    
 1.0e-18    
 1.0e-15    
 1.0e-12    
 1.0e-9     
 1.0e-6     
 0.001      

In [69]:
log10(1e-50)

-50.0

In [31]:
g(2/3 + ϵ)

15-element Array{Float64,1}:
 9.97604e-46
 9.97604e-43
 1.00093e-39
 9.99934e-37
 1.0e-33    
 1.0e-30    
 1.0e-27    
 1.0e-24    
 1.0e-21    
 1.0e-18    
 1.0e-15    
 1.0e-12    
 1.0e-9     
 1.0e-6     
 0.001      

In [22]:
y = f(2/3+ϵ)

15-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 8.88178e-16
 9.99978e-13
 1.0e-9     
 1.0e-6     
 0.001      

In [74]:
log10.(g(2/3+ϵ) + 1e-20)

15-element Array{Float64,1}:
 -20.0   
 -20.0   
 -20.0   
 -20.0   
 -20.0   
 -20.0   
 -20.0   
 -20.0   
 -19.9586
 -17.9957
 -15.0   
 -12.0   
  -9.0   
  -6.0   
  -3.0   

In [76]:
log10.(ϵ)

15-element Array{Float64,1}:
 -15.0
 -14.0
 -13.0
 -12.0
 -11.0
 -10.0
  -9.0
  -8.0
  -7.0
  -6.0
  -5.0
  -4.0
  -3.0
  -2.0
  -1.0

In [79]:
plot(log10.(ϵ), log10.(f(2/3+ϵ) + 1e-50), label="f(x)")
plot!(log10.(ϵ), log10.(g(2/3+ϵ) + 1e-50), label="g(x)")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 -10 
 
 
 -5 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 
 
 
 
 
 f(x) 
 
 
 
 g(x)

In [34]:
secantsearch(g,0.0,1.0,true)

n = 1, c = 0.8888888888888888, f(c) = 0.010973936899862825
n = 2, c = 0.8421052631578947, f(c) = 0.005399772129616129
n = 3, c = 0.7967853042479908, f(c) = 0.0022030204162444673
n = 4, c = 0.7655533526628064, f(c) = 0.0009669710403883907
n = 5, c = 0.7411203540240837, f(c) = 0.00041272296404358126
n = 6, c = 0.7229262275146163, f(c) = 0.00017806928441267748
n = 7, c = 0.7091194332075736, f(c) = 7.650996304061845e-5
n = 8, c = 0.6987180509624981, f(c) = 3.292610616566885e-5
n = 9, c = 0.690860164106507, f(c) = 1.4161066591496093e-5
n = 10, c = 0.6849301974471621, f(c) = 6.091920437954204e-6
n = 11, c = 0.680453281996499, f(c) = 2.6204324846269565e-6
n = 12, c = 0.6770739084150347, f(c) = 1.127215439109328e-6
n = 13, c = 0.6745228512281511, f(c) = 4.848808504160482e-7
n = 14, c = 0.6725971277811974, f(c) = 2.0857650592181485e-7
n = 15, c = 0.6711434384855313, f(c) = 8.972115936668311e-8
n = 16, c = 0.6700460819698477, f(c) = 3.859443603535173e-8
n = 17, c = 0.6692177117342263, f(c) = 1.6

(0.6666788670391179, [1.0, 0.888889, 0.842105, 0.796785, 0.765553, 0.74112, 0.722926, 0.709119, 0.698718, 0.69086  …  0.66682, 0.666782, 0.666754, 0.666733, 0.666716, 0.666704, 0.666695, 0.666688, 0.666683, 0.666679])

In [62]:
ϵ = linspace(0.00001,0.0001)
x = 2/3 + ϵ
plot(x,f(x), marker=:circ, label="f(x)")
plot!(x,g(x),label="g(x)")
plot!(xlabel="x", yscale=:log10, xscale=:linear)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.666700 
 
 
 0.666725 
 
 
 0.666750 
 
 
 10^-15 
 
 
 10^-14 
 
 
 10^-13 
 
 
 10^-12 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 405.077,360.065 405.078,360.065 405.079,360.065 405.08,360.065 405.081,360.065 405.082,360.065 405.083,360.065 405.084,360.065 405.085,360.065 405.086,360.065 
 405.087,360.065 405.088,360.065 405.089,360.065 405.09,360.065 405.091,360.065 405.092,360.065 405.093,360.065 405.094,360.065 405.095,360.065 405.096,360.065 
 405.097,360.065 405.098,360.065 405.099,360.065 405.1,360.065 405.101,360.065 405.102,360.065 405.103,360.065 405.103,360.065 405.104,360.065 405.105,360.065 
 405.106,360.065 405.107,360.065 405.108,360.065 405.109,360.065 405.11,360.065 405.111,360.065 405.112,360.065 405.113,360.065 405.114,360.065 405.115,360.065 
 405.116,360.065 405.117,360.065 405.118,360.065 405.119,360.065 405.12,360.065 405.121,360.065 405.122,360.065 405.123,360.065 405.124,360.065 405.125,360.065 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 405.077,360.065 405.078,360.065 405.079,360.065 405.08,360.065 405.081,360.065 405.082,360.065 405.083,360.065 405.084,360.065 405.085,360.065 405.086,360.065 
 405.087,360.065 405.088,360.065 405.089,360.065 405.09,360.065 405.091,360.065 405.092,360.065 405.093,360.065 405.094,360.065 405.095,360.065 405.096,360.065 
 405.097,360.065 405.098,360.065 405.099,360.065 405.1,360.065 405.101,360.065 405.102,360.065 405.103,360.065 405.103,360.065 405.104,360.065 405.105,360.065 
 405.106,360.065 405.107,360.065 405.108,360.065 405.109,360.065 405.11,360.065 405.111,360.065 405.112,360.065 405.113,360.065 405.114,360.065 405.115,360.065 
 405.116,360.065 405.117,360.065 405.118,360.065 405.119,360.065 405.12,360.065 405.121,360.065 405.122,360.065 405.123,360.065 405.124,360.065 405.125,360.065 
 
 "/>
 
 
 
 
 
 
 f(x) 
 
 
 
 g(x)

GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine SET_WINDOW


In [47]:
x = 2/3 + ϵ

0.6667666666666666:1.836734693877551e-5:0.6676666666666666

In [50]:
plot(x,f(x), marker=:circ, label="f(x)")
plot!(x,g(x),label="g(x)")
plot!(xlabel="x", yscale=:log10)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.6670 
 
 
 0.6675 
 
 
 10^-12 
 
 
 10^-11 
 
 
 10^-10 
 
 
 10^-9 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 69.4657,360.065 79.5875,334.567 89.7093,312.743 99.8312,293.681 109.953,276.755 120.075,261.533 130.197,247.703 140.319,235.034 150.44,223.344 160.562,212.493 
 170.684,202.369 180.806,192.88 190.928,183.951 201.05,175.521 211.171,167.535 221.293,159.95 231.415,152.728 241.537,145.835 251.659,139.242 261.781,132.924 
 271.902,126.86 282.024,121.03 292.146,115.416 302.268,110.003 312.39,104.777 322.512,99.726 332.634,94.838 342.755,90.103 352.877,85.5117 362.999,81.0558 
 373.121,76.7273 383.243,72.5193 393.365,68.4252 403.486,64.4391 413.608,60.5552 423.73,56.7687 433.852,53.0746 443.974,49.4686 454.096,45.9466 464.217,42.5047 
 474.339,39.1394 484.461,35.8474 494.583,32.6255 504.705,29.4708 514.827,26.3806 524.948,23.3522 535.07,20.3833 545.192,17.4716 555.314,14.6148 565.436,11.811 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 69.4657,360.064 79.5875,334.561 89.7093,312.744 99.8312,293.681 109.953,276.754 120.075,261.532 130.197,247.703 140.319,235.034 150.44,223.344 160.562,212.493 
 170.684,202.369 180.806,192.88 190.928,183.951 201.05,175.521 211.171,167.535 221.293,159.95 231.415,152.728 241.537,145.834 251.659,139.242 261.781,132.924 
 271.902,126.86 282.024,121.03 292.146,115.416 302.268,110.003 312.39,104.777 322.512,99.726 332.634,94.838 342.755,90.103 352.877,85.5117 362.999,81.0558 
 373.121,76.7273 383.243,72.5193 393.365,68.4252 403.486,64.4391 413.608,60.5552 423.73,56.7687 433.852,53.0746 443.974,49.4686 454.096,45.9466 464.217,42.5047 
 474.339,39.1394 484.461,35.8474 494.583,32.6255 504.705,29.4708 514.827,26.3806 524.948,23.3522 535.07,20.3833 545.192,17.4716 555.314,14.6148 565.436,11.811 
 
 "/>
 
 
 
 
 
 
 f(x) 
 
 
 
 g(x)